<a href="https://colab.research.google.com/github/ankishore/ann_churn_modelling/blob/main/ann_churn_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
tf.__version__

'2.8.0'

# Data preprocessing


### Import data set

In [4]:
dataset = pd.read_csv('https://raw.githubusercontent.com/ankishore/ann_churn_modelling/main/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [5]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [6]:
print(y)

[1 0 1 ... 1 1 0]


### Encode categorial data

Encode gender

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [8]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One hot encode countries

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [10]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Split dataset into training and test set

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature scaling

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Building ANN

Initialize

In [13]:
ann = tf.keras.models.Sequential()

Add input layer and first hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

Add second hidden layer

In [15]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

Add output layer

In [16]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Compiling

In [17]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training on train dataset

In [18]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6283 - accuracy: 0.7310
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4894 - accuracy: 0.7984
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4536 - accuracy: 0.8012
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4365 - accuracy: 0.8087
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4235 - accuracy: 0.8206
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4128 - accuracy: 0.8280
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4054 - accuracy: 0.8304
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3979 - accuracy: 0.8324
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3908 - accuracy: 0.8354
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3853 - accura

# Predict

Predict on a custom input

In [19]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

[[0.03050628]]


Predict on test dataset and compare results

In [24]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Display stats

In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
accuracy_score(y_test, y_pred)

[[1516   79]
 [ 199  206]]


0.861